# **Other elements of Keras**

## Building Complex Models Using the **Functional API**
Aunque los `Sequential models` son extremadamente comunes, a veces
es útil construir redes neuronales con topologías más complejas, o con
múltiples entradas o salidas. Para ello, Keras ofrece la `API Funcional`,un
ejemplo de red neuronal no secuencial es **Wide & Deep** neural
network.

  Esta arquitectura, conecta todas o parte de las
entradas directamente a la capa de salida, lo que hace posible que la red neuronal aprenda tanto
patrones profundos (utilizando la ruta profunda) como reglas simples (a
través de la ruta corta). Por el contrario, un **MLP** normal obliga a que
todos los datos fluyan a través de toda la pila de capas; así, los patrones simples de los
datos pueden acabar distorsionados por esta secuencia de transformaciones.
___

Let's load, split and scale the California housing dataset(because SDG will be used)
 
 Para simplificar,utilizaremos la API de Scikit-Learn `fetch_california_housing()` ya que este conjunto de datos sólo contiene características numéricas (no existe la característica
proximidad_del_océano) y no hay valores perdidos

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)


Now let's create the `Wide & Deep ANN` using **API Funtional** 

In [26]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Input, concatenate
from keras import Model
from keras.backend import clear_session
import numpy as np

# reiniciar (en caso de q ya se hayan ejecutados otras capas) la genracion del # de los sequential
clear_session()

# estableces las semillas para lograr reproducir los mismos resultados
np.random.seed(42)
tf.random.set_seed(42)

input_ = Input(shape=X_train.shape[1:])
hidden1 = Dense(30, activation="relu")(input_)
hidden2 = Dense(30, activation="relu")(hidden1)
concat = concatenate([input_, hidden2])
output = Dense(1)(concat)
model = Model(inputs=[input_], outputs=[output])


Repasemos cada línea de este código:
* En primer lugar, se crea un objeto **Input**, se trata de
una especificación del tipo de entrada que recibirá el modelo, incluyendo su *shape and dtype*
* Luego se crea una capa **Dense** con 30 neuronas,
utilizando la función de activación ReLU. Nada más crearla,
observa que la llamamos como una función, pasándole la 
entrada. Por eso se llama API Funcional. Nótese que sólo le
se le esta indicando a Keras cómo debe conectar las capas entre sí;
aún no se están procesando datos reales.

* A continuación creamos una segunda capa oculta, y de nuevo la
utilizamos como función. Nótese que le pasamos la salida de la
primera capa oculta.
* Después se crea una capa **Concatenate**, y una vez más
se una inmediatamente como una función, para concatenar la
entrada y la salida de la segunda capa oculta. 

* La ultima capa es la de salida, con una sola neurona y
sin función de activación, y la llamamos como una función,
pasándole el resultado de la concatenación.
* Finalmente, se crea un Modelo Keras, especificando qué entradas
y salidas utilizar.

Una vez que has construido el modelo Keras, todo es exactamente igual que
antes, así que no hay necesidad de repetirlo aquí: debes compilar el
modelo, entrenarlo, evaluarlo y usarlo para hacer predicciones.

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 30)           270         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 30)           930         ['dense[0][0]']                  
                                                                                                  
 concatenate (Concatenate)      (None, 38)           0           ['input_1[0][0]',                
                                                                  'dense_1[0][0]']            

In [27]:
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(
    learning_rate=1e-3), metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)


Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 2.0584 - accuracy: 0.0027 - val_loss: 0.7315 - val_accuracy: 0.0044
Epoch 2/20
363/363 [==============================] - 2s 4ms/step - loss: 0.6249 - accuracy: 0.0029 - val_loss: 0.6326 - val_accuracy: 0.0044
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5876 - accuracy: 0.0029 - val_loss: 0.5792 - val_accuracy: 0.0044
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5580 - accuracy: 0.0029 - val_loss: 0.5248 - val_accuracy: 0.0044
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5354 - accuracy: 0.0029 - val_loss: 0.5000 - val_accuracy: 0.0044
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5141 - accuracy: 0.0029 - val_loss: 0.4893 - val_accuracy: 0.0044
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4985 - accuracy: 0.0029 - val_loss: 0.4905 - val_accuracy: 0.0044
Epoch 

In [30]:
model.predict(X_test[:1])

1/1 [==============================] - 0s 95ms/step


array([[0.6457891]], dtype=float32)

What if you want to send different subsets of input features through the wide or deep paths?
 
We will send 5 features (features 0 to 4), and 6 through the deep path (features 2 to 7). Note that 3 features will go through both (features 2, 3 and 4).

In [31]:
input_A = Input(shape=[5], name="wide_input")
input_B = Input(shape=[6], name="deep_input")
hidden1 = Dense(30, activation="relu")(input_B)
hidden2 = Dense(30, activation="relu")(hidden1)
concat = concatenate([input_A, hidden2])
output = Dense(1, name="output")(concat)

model = Model(inputs=[input_A, input_B], outputs=[output])


In [32]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 deep_input (InputLayer)        [(None, 6)]          0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 30)           210         ['deep_input[0][0]']             
                                                                                                  
 wide_input (InputLayer)        [(None, 5)]          0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 30)           930         ['dense_3[0][0]']                
                                                                                            

In [33]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

# debemos pasar un par de matrices `(X_train_A, X_train_B)`: una por entrada. 
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))


Epoch 1/20
363/363 [==============================] - 2s 3ms/step - loss: 2.3838 - val_loss: 2.1185
Epoch 2/20
363/363 [==============================] - 2s 4ms/step - loss: 0.8630 - val_loss: 0.7853
Epoch 3/20
363/363 [==============================] - 2s 4ms/step - loss: 0.6834 - val_loss: 0.6412
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.6213 - val_loss: 0.6059
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5849 - val_loss: 0.5735
Epoch 6/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5573 - val_loss: 0.5535
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5349 - val_loss: 0.5296
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: 0.5168 - val_loss: 0.5021
Epoch 9/20
363/363 [==============================] - 2s 4ms/step - loss: 0.5010 - val_loss: 0.4810
Epoch 10/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4877 - val_loss: 0.4632

### Multiple outputs

Es posible que desee tener **multiple outputs** por multiples casos como pueden ser:
* La tarea puede exigirlo. Por ejemplo, puede que desee localizar y
clasificar el objeto principal de una imagen. Se trata tanto de una
tarea de regresión (encontrar las coordenadas del centro del
objeto, así como su anchura y altura) como de una tarea de
clasificación.
* Del mismo modo, puede tener varias tareas independientes
basadas en los mismos datos. Claro que podría entrenar una red
neuronal por tarea, pero en muchos casos obtendrá mejores
resultados en todas las tareas entrenando una sola red neuronal
con una salida por tarea. Esto se debe a que la red neuronal
puede aprender características de los datos que son útiles en
todas las tareas. Por ejemplo, puede realizar una clasificación
multitarea de imágenes de caras, utilizando una salida para
clasificar la expresión facial de la persona (sonriente, sorprendida,
etc.) y otra salida para identificar si lleva gafas o no.

* Otro caso de uso es como técnica de regularización (es decir, una
restricción de entrenamiento cuyo objetivo es reducir el
sobreajuste y mejorar así la capacidad de generalización del
modelo). Por ejemplo, es posible que desee añadir algunas
salidas auxiliares en una arquitectura de red neuronal para garantizar que 
la parte subyacente de la red aprenda algo útil por sí misma,
 sin depender del resto de la red.
  
![alt](resources/multi-outputsANN.png)


Adding an auxiliary output for regularization (builds the networks represented in *Figure 10-16*):

In [34]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

Cada salida necesitará su propia función de pérdida. Por lo tanto, cuando
*se compile* el modelo, se debe pasar una lista de pérdidas (si
se pasa una sola pérdida, Keras asumirá que se debe usar la misma
pérdida para todas las salidas). Por defecto, Keras calculará todas estas
`loss` y simplemente las sumará para obtener la pérdida final utilizada
para el entrenamiento. Como interesa  mucho más  la salida
principal que por la salida auxiliar (ya que sólo se utiliza para la
regularización), se debe dar a la pérdida de la salida
principal un peso mucho mayor. Afortunadamente, es posible establecer
todos los pesos de pérdida al compilar el modelo como se muestra 

In [35]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [36]:
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=20,
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 2s 4ms/step - loss: 2.1556 - main_output_loss: 1.8355 - aux_output_loss: 5.0370 - val_loss: 1.4597 - val_main_output_loss: 1.0725 - val_aux_output_loss: 4.9443
Epoch 2/20
363/363 [==============================] - 2s 5ms/step - loss: 0.9820 - main_output_loss: 0.7320 - aux_output_loss: 3.2320 - val_loss: 1.0373 - val_main_output_loss: 0.6526 - val_aux_output_loss: 4.5000
Epoch 3/20
363/363 [==============================] - 2s 5ms/step - loss: 0.7895 - main_output_loss: 0.6304 - aux_output_loss: 2.2212 - val_loss: 0.9979 - val_main_output_loss: 0.6089 - val_aux_output_loss: 4.4993
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7000 - main_output_loss: 0.5848 - aux_output_loss: 1.7363 - val_loss: 0.9525 - val_main_output_loss: 0.5637 - val_aux_output_loss: 4.4517
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6488 - main_output_loss: 0.5540 - aux_output_loss: 1.5019 - val_loss

In [40]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])

y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])


1/1 [==============================] - 0s 27ms/step


##  Subclassing API to Build Dynamic Models

Tanto la `API Sequential` como la `API Funtional`  son declarativas: se empieza por declarar qué capas se desean utilizar y cómo deben conectar, y sólo entonces se puede empezar a alimentar el modelo con
algunos datos para el entrenamiento o la inferencia. Esto tiene muchas
ventajas: el modelo puede guardarse, clonarse y compartirse fácilmente; su
estructura puede visualizarse y analizarse; el marco puede inferir formas y
comprobar tipos, por lo que los errores pueden detectarse pronto (es decir,
antes de que ningún dato pase por el modelo). También es bastante fácil de
depurar, ya que todo el modelo es un gráfico estático de capas. Pero la otra
cara de la moneda es simplemente eso: es estático.
 
Algunos modelos implican bucles, formas
variables, bifurcaciones condicionales y otros comportamientos dinámicos.
Para estos casos, o simplemente si prefiere un estilo de programación
más imperativo, la `Subclassing API` es para usted. Basta con subclasificar la clase Model, crear las capas que necesite en elconstructor y utilizarlas para realizar los cálculos que desee en el método
`call()`. 


Esta flexibilidad extra tiene un coste: la arquitectura de tu modelo está
oculta dentro del método call(), por lo que Keras no puede inspeccionarlo
fácilmente; no puede guardarlo o clonarlo; y cuando llamas al método
summary(), sólo obtienes una lista de capas, sin ninguna información sobre
cómo están conectadas entre sí. Además, Keras no puede comprobar los
tipos y formas de antemano, y por lo tanto mas fácil cometer errores
___
 Por ejemplo, creando una instancia de la siguiente clase
**WideAndDeepModel** obtenemos un modelo equivalente al que acabamos
de construir con la API Funcional. A continuación, puede compilarlo,
evaluarlo y utilizarlo para hacer predicciones, exactamente como
acabamos de hacer

In [41]:
class WideAndDeepModel(Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = Dense(units, activation=activation)
        self.hidden2 = Dense(units, activation=activation)
        self.main_output = Dense(1)
        self.aux_output = Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output


model = WideAndDeepModel(30, activation="relu")


In [42]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/10
363/363 [==============================] - 2s 4ms/step - loss: 2.9256 - output_1_loss: 2.6530 - output_2_loss: 5.3792 - val_loss: 1.8043 - val_output_1_loss: 1.2635 - val_output_2_loss: 6.6716
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 1.1196 - output_1_loss: 0.8814 - output_2_loss: 3.2635 - val_loss: 1.1044 - val_output_1_loss: 0.8003 - val_output_2_loss: 3.8410
Epoch 3/10
363/363 [==============================] - 2s 5ms/step - loss: 0.8413 - output_1_loss: 0.6978 - output_2_loss: 2.1334 - val_loss: 0.8301 - val_output_1_loss: 0.6333 - val_output_2_loss: 2.6013
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.7286 - output_1_loss: 0.6304 - output_2_loss: 1.6125 - val_loss: 0.7218 - val_output_1_loss: 0.5825 - val_output_2_loss: 1.9749
Epoch 5/10
363/363 [==============================] - 1s 2ms/step - loss: 0.6666 - output_1_loss: 0.5894 - output_2_loss: 1.3617 - val_loss: 0.6550 - val_output_1_loss: 0.5440 - val_ou

## Saving and Restoring

Cuando se utiliza la `API Secuencial` o la `API Funcional`, guardar un
modelo Keras entrenado es de lo más sencillo:
```python 
    model = keras.layers.Sequential([...]) # o keras.Model([...])
    model.compile([...])
    model.fit([...])
    model.save("mi_modelo_keras.h5")
    
    #cargar en otro scripts
    keras.models.load_model("mi_modelo_keras.h5")
```

Keras utilizará el formato HDF5 para guardar tanto la arquitectura del
modelo (incluyendo los hiperparámetros de cada capa) como los valores
de todos los parámetros del modelo para cada capa (por ejemplo, pesos
de conexión y sesgos). También guarda el optimizador (incluyendo sus
hiperparámetros y cualquier estado que pueda tener).
 
Desafortunadamente  cuando se utilice la `Subclassing`  no se puede guardar fácilmente el modelo, pero se puede utilizar **save_weights()** y **load_weights()** para al menos guardar y restaurar los parámetros del modelo, pero tendrá que guardar y restaurar todo lo demás usted mismo
```python 
    model.save_weights("my_keras_weights.ckpt")
    model.load_weights("my_keras_weights.ckpt")
```

In [46]:
model = keras.models.Sequential([
    Dense(30, activation="relu", input_shape=[8]),
    Dense(30, activation="relu"),
    Dense(1)
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))


Epoch 1/3
363/363 [==============================] - 2s 6ms/step - loss: 1.9701 - val_loss: 0.7871
Epoch 2/3
363/363 [==============================] - 1s 4ms/step - loss: 0.7556 - val_loss: 0.8961
Epoch 3/3
363/363 [==============================] - 3s 7ms/step - loss: 0.6958 - val_loss: 0.7924


In [48]:
model.save("resources/my_keras_model.h5")

In [50]:
from keras.models import  load_model
model_restaured = load_model("resources/my_keras_model.h5")

In [53]:
model_restaured.predict(X_test[:1])

1/1 [==============================] - 0s 44ms/step


array([[0.9575483]], dtype=float32)

# Using Callbacks during Training(for examle: *EarlyStop*)

In [71]:
clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    

El método fit() acepta un argumento `callbacks` que te permite especificar
una lista de objetos que Keras llamará al inicio y al final del
entrenamiento, al inicio y al final de cada epoch, e incluso antes y
después de procesar cada lote. Por ejemplo, el callback **ModelCheckpoint**
guarda puntos de control de tu modelo a intervalos regulares durante el
entrenamiento, por defecto al final de cada epoch.

Si utiliza un conjunto de validación durante el entrenamiento,
puede establecer **save_best_only=True** al crear el **ModelCheckpoint**. En
este caso, sólo guardará su modelo cuando su rendimiento en el
conjunto de validación sea el mejor hasta el momento. De esta forma, no
tendrá que preocuparse de entrenar durante demasiado tiempo y
sobreajustar el conjunto de entrenamiento: simplemente restaure el último
modelo guardado después del entrenamiento, y éste será el mejor modelo
en el conjunto de validación. El siguiente código es una forma sencilla de
aplicar `Early Stopping`

In [92]:
from keras.callbacks import ModelCheckpoint

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

checkpoint_cb = ModelCheckpoint("resources/my_keras_model_early_stop.h5", save_best_only=True)


history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])
model = load_model("resources/my_keras_model_early_stop.h5")  # rollback to best model
mse_test = model.evaluate(X_test, y_test)


Epoch 1/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3261 - val_loss: 0.3179
Epoch 2/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3261 - val_loss: 0.3170
Epoch 3/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3255 - val_loss: 0.3203
Epoch 4/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3258 - val_loss: 0.3483
Epoch 5/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3256 - val_loss: 0.3324
Epoch 6/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3251 - val_loss: 0.3242
Epoch 7/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3247 - val_loss: 0.3122
Epoch 8/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3242 - val_loss: 0.3142
Epoch 9/10
363/363 [==============================] - 1s 2ms/step - loss: 0.3239 - val_loss: 0.3173
Epoch 10/10
162/162 [==============================] - 0s 1ms/step - loss: 0.3288


Otra forma de aplicar `Early Stopping` es utilizar simplemente la callback EarlyStopping. Interrumpirá el entrenamiento cuando no mida ningún progreso en el conjunto de validación durante
un número de épocas (definido por el argumento paciencia), y opcionalmente volverá al mejor modelo.
 
En este caso, no hay necesidad de restaurar el mejor modelo
guardado porque la llamada de retorno EarlyStopping mantendrá un
registro de los mejores pesos y los restaurará por ti al final del
entrenamiento.

In [90]:
from keras.callbacks import EarlyStopping

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])
mse_test = model.evaluate(X_test, y_test)


Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3339 - val_loss: 0.3316
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3338 - val_loss: 0.3466
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3332 - val_loss: 0.3311
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3331 - val_loss: 0.3217
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3326 - val_loss: 0.3223
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3317 - val_loss: 0.3184
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3315 - val_loss: 0.3153
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3308 - val_loss: 0.3176
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3304 - val_loss: 0.3182
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3299 - val_lo

In [91]:
len(history.history['loss'])

29

Si necesita un control adicional(por ejemplo aumentar learning_rate), puede escribir fácilmente sus propias
callbacks. Como ejemplo de cómo hacerlo, la
siguiente llamada de retorno personalizada mostrará la relación entre la
pérdida de validación y la pérdida de entrenamiento durante el
entrenamiento (por ejemplo, para detectar el sobreajuste)

In [96]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

In [99]:
val_train_ratio_cb = PrintValTrainRatioCallback()
history = model.fit(X_train, y_train, epochs=1,
                    validation_data=(X_valid, y_valid),
                    callbacks=[val_train_ratio_cb])

337/363 [==========================>...] - ETA: 0s - loss: 0.3211
val/train: 0.96
363/363 [==============================] - 1s 2ms/step - loss: 0.3222 - val_loss: 0.3092


Para el entrenamiento debe implementar **on_train_begin(), on_train_end(), on_epoch_begin(), on_epoch_end(),on_batch_begin(), y on_batch_end()**. 

 Para la evaluación, debe implementar **on_test_begin(), on_test_end(),on_test_batch_begin(), o on_test_batch_end()** (llamado por evaluate()), 
 
Para la predicción debe implementar
**on_predict_begin(),on_predict_end(), on_predict_batch_begin(), o on_predict_batch_end()**
(llamado por predict()).